In [ ]:
import pathlib
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
from torchsummary import summary
from matplotlib import rc

from siammot.modelling.track_head.EMM.attention import DeformableSiameseAttention

%matplotlib inline

%load_ext autoreload
%autoreload 2

font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 12}
rc('font', **font)

In [ ]:
batch_size = 4
n_channels = 128
n_query_key_channels = n_channels // 8
template_size = 15
sr_size = int(template_size * 2.0)

template_features = torch.rand((batch_size, n_channels, template_size, template_size))
sr_features = torch.rand((batch_size, n_channels, sr_size, sr_size))

attention = DeformableSiameseAttention(n_channels, n_query_key_channels)
attentional_template_features, attentional_sr_features = attention(template_features, sr_features)
attentional_template_features.shape, attentional_sr_features.shape

In [ ]:
summary(attention, [(n_channels, template_size, template_size), (n_channels, sr_size, sr_size)])